# Scraper

Use `base` env

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd
import numpy as np
import urllib
import gcsfs

In [56]:
url = 'https://www.morphmarket.com/us/c/reptiles/pythons/ball-pythons?view=grid'

options = webdriver.ChromeOptions()
# Config to work on Google Colab
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')

# Pretend to be a non-headless browser
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36")
options.add_argument("--lang=en-US")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--window-size=1280,720")
driver = webdriver.Chrome(options=options)
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
soup

<html class="no-js fontawesome-i2svg-active fontawesome-i2svg-complete" lang=""><head><meta content="Az520Inasey3TAyqLyojQa8MnmCALSEU29yQFW8dePZ7xQTvSt73pHazLFTK5f7SyLUJSo2uKLesEtEa9aUYcgMAAACPeyJvcmlnaW4iOiJodHRwczovL2dvb2dsZS5jb206NDQzIiwiZmVhdHVyZSI6IkRpc2FibGVUaGlyZFBhcnR5U3RvcmFnZVBhcnRpdGlvbmluZyIsImV4cGlyeSI6MTcyNTQwNzk5OSwiaXNTdWJkb21haW4iOnRydWUsImlzVGhpcmRQYXJ0eSI6dHJ1ZX0=" http-equiv="origin-trial"/>
<!-- Google Tag Manager -->
<script async="" crossorigin="anonymous" src="https://connect.facebook.net/en_US/sdk.js?hash=07dbea6442f5667afb22bbfb18381f3b"></script><script async="" crossorigin="anonymous" integrity="sha384-0lJkOVHDy3ItYlCbUoEzThjP3hLhLYfEFPAkVOCxnJpb5K9Fllso+S8TRBILcfPo" src="https://www.gstatic.com/recaptcha/releases/V6_85qpc2Xf2sbe3xTnRte7m/recaptcha__en.js" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=AW-943812589&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script async="" src="//www.google-a

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [54]:
url = 'https://www.morphmarket.com/us/c/reptiles/pythons/ball-pythons?view=grid&page=1'

In [93]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
# options.add_argument('--no-sandbox')
driver = webdriver.Chrome(options=options)

# IP check
url_check_ip = 'https://api.ipify.org/?format=json'
driver.get(url_check_ip)
page_source = driver.page_source
soup = BeautifulSoup(page_source, "html.parser")
ip_address = soup.find('body').text
print(ip_address)

# Try first page
driver.get(url)
time.sleep(5) # Loading page might take time
html = driver.page_source
driver.quit()

{"ip":"34.29.234.168"}


## Scrape URLs

In [94]:
soup = BeautifulSoup(html, 'html.parser')
# soup

In [129]:
def scrape_urls(url_prefix, start_num, end_num):
    urls = []
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    # options.add_argument('--no-sandbox')
    driver = webdriver.Chrome(options=options)

    for n in range(start_num, end_num):
        url_length = len(urls)

        url = url_prefix + str(n)
        driver.get(url)
        time.sleep(4) # Loading page might take time
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        result_container = soup.find_all('a', class_='animalCard--avL0R')
        urls.extend([a.get('href') for a in result_container if a.get('href')])
        assert len(urls) > url_length, 'Nothing is returned'

        print(f'page {n} complete')

        if n % 10 == 0:
            pd.DataFrame(urls, columns=['url']).to_csv('data/url_df.csv', index=False)
    
    driver.quit()
    return len(urls)



In [124]:
start_num = 1
end_num = 10
url_prefix = 'https://www.morphmarket.com/us/c/reptiles/pythons/ball-pythons?view=grid&page='
urls = scrape_urls(url_prefix, start_num, end_num)

page 1 complete
page 2 complete
page 3 complete
page 4 complete
page 5 complete
page 6 complete
page 7 complete
page 8 complete
page 9 complete


In [126]:
url_df = pd.DataFrame(urls, columns=['url']).to_csv('data/url_df.csv', index=False)
url_df

In [96]:
urls = []
result_container = soup.find_all('a', class_='animalCard--avL0R')
urls = [a.get('href') for a in result_container if a.get('href')]
print(len(urls))
urls[0:4]

24


['/us/c/reptiles/pythons/ball-pythons/2265731',
 '/us/c/reptiles/pythons/ball-pythons/2123926',
 '/us/c/reptiles/pythons/ball-pythons/1669074',
 '/us/c/reptiles/pythons/ball-pythons/2168560']

In [103]:
pages = soup.find('ul', class_="MuiPagination-ul css-nhb8h9") # Pagination class

# Find all the page buttons
pages_button = pages.find_all('button', class_="MuiButtonBase-root MuiPaginationItem-root MuiPaginationItem-sizeMedium MuiPaginationItem-text MuiPaginationItem-circular MuiPaginationItem-page css-115r0jl")
last_page_num = int(pages_button[-1].text.strip())
last_page_num

1543

# Scrape data

In [3]:
complete_url = pd.read_csv('data/complete_url.csv')
complete_url.shape

(35527, 1)

In [4]:
complete_url.iloc[0:5, 0]

0    /us/c/reptiles/pythons/ball-pythons/1622444
1    /us/c/reptiles/pythons/ball-pythons/1970200
2     /us/c/reptiles/pythons/ball-pythons/933034
3     /us/c/reptiles/pythons/ball-pythons/860291
4    /us/c/reptiles/pythons/ball-pythons/1669137
Name: url, dtype: object

In [7]:
# fs = gcsfs.GCSFileSystem(project='ubc-mds-pool')
# with fs.open('gs://sam_datastorage/complete_url.csv', 'w') as f:
#     complete_url.to_csv(f)

In [19]:
test_url = 'https://www.morphmarket.com' + complete_url.iloc[5,0]
# test_url = 'https://www.morphmarket.com/us/c/reptiles/pythons/ball-pythons/2074211'
print(test_url)

options = webdriver.ChromeOptions()
options.add_argument('--headless')
# options.add_argument('--disable-gpu')
# options.add_argument('--no-sandbox')
driver = webdriver.Chrome(options=options)

driver.get(test_url)
time.sleep(4) # Loading page might take time
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

https://www.morphmarket.com/us/c/reptiles/pythons/ball-pythons/1983770


In [20]:
pic_container = soup.find_all('img', class_='thumbCarouselImage--RFBqw')
pic_url = [img['srcset'].split(', ')[0].split(' ')[0] for img in pic_container if 'srcset' in img.attrs]
if pic_url == []:
    pic_url = soup.find_all('source')[0]['srcset'].split(' ')[0]
[pic_url]

[['https://d20zyr0oabjxc2.cloudfront.net/variants/media/raw_images/gray_family_snakes/2023/11/20231103224548.500-2023-01-08-856x856h.jpeg?signature=FF2EFBF3B551FD84F3F0A3E04F05CF6DB9E75CF5D576C4F8B031E11276B18F59',
  'https://d20zyr0oabjxc2.cloudfront.net/variants/media/raw_images/gray_family_snakes/2023/11/20231103224548.587-2023-01-08b-856x856h.jpeg?signature=CC06ABC95EC606A060671943540A585EFB4320C9ADE910F31B22B12FF30F5133',
  'https://d20zyr0oabjxc2.cloudfront.net/variants/media/raw_images/gray_family_snakes/2023/11/20231103224547.303-2023-01Clutch-856x856h.jpeg?signature=04EA7254202A968C7ADC213196D4293E9E5A033C7D476E3AEA13E9521C87ED8B']]

In [80]:
urllib.request.urlretrieve(pic_url[0], "data/img/first.png")

('data/img/first.png', <http.client.HTTPMessage at 0x7f2a2e05d510>)

In [30]:
def get_gene(soup):
    # Check if the container exists
    container = soup.find('div', class_='container--N6XSR')
    if not container:
        return []  # or return ['unknown'] if you prefer to indicate no genes found

    gene_classes = {
        'dom-codom': 'labelBadge--oeqO_ dom-codom--KcmpO',
        'het-rec': 'labelBadge--oeqO_ het-rec--Z8INA',
        'vis-rec': 'labelBadge--oeqO_ vis-rec--YbPYK',
        'pos-rec': 'labelBadge--oeqO_ pos-rec--TteXe',
        'polygenic': 'labelBadge--oeqO_ polygenic--GxLqN',
        'super-dom-codom': 'labelBadge--oeqO_ super-dom-codom--iEgCD'
    }
    
    # Gather all relevant gene traits
    gene_data = []
    for gene_type, class_name in gene_classes.items():
        # Find all span elements for each gene type based on its class name
        genes = {trait.get_text() for trait in container.find_all('span', class_=class_name)}
        gene_data.extend(genes)  # Add to the gene list, using extend to add each item in the set

    return gene_data
get_gene(soup)

['Pastel', 'Het Axanthic (VPI)', 'Het Desert Ghost', 'Het Piebald']

In [29]:
def get_origin(soup):
    classes_to_check = [
        'labelBadge--oeqO_ warning--WSnpA',
        'labelBadge--oeqO_ success--vMoO7',
        'labelBadge--oeqO_ danger--aAl4n'
    ]
    
    for class_name in classes_to_check:
        origin = [trait.get_text() for trait in soup.find_all('span', class_=class_name)]
        if origin:
            return origin[0]
        
    return 'unknown'

get_origin(soup)

'Self Produced'

In [23]:
def get_sex(soup):
    sex = 'unknown' # Default as unknown
    if soup.find_all('svg', class_='svg-inline--fa fa-mars sex male') != []:
        sex = 'male'
    if soup.find_all('svg', class_='svg-inline--fa fa-venus sex female') != []:
        sex = 'female'
    if soup.find_all('svg', class_='svg-inline--fa fa-venus-mars sex mixed') != []:
        sex = 'mixed'
    return sex

get_sex(soup)


'female'

In [26]:
# Price

def get_price(soup):
    try:
        price = float(soup.find_all('h1', class_='salePrice--qNIIs')[0].get_text().replace("$", "").replace(",", ""))
    except:
        price = 0
    return price

get_price(soup)

1490.0

In [27]:
def get_birth(soup):
    try:
        birth_divs = [div for div in soup.find_all('div', class_='labelValueContainer--z1CP3')
                if div.find('b', string='Birth:')]
        birth_dates = [div.find('span').text for div in birth_divs]
        birth = birth_dates[0]
    except:
        birth = ''
    return birth

get_birth(soup)

'7th September 2023'

In [12]:
def scrape_pic(start_num, end_num):
    dict = {}
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)
    fs = gcsfs.GCSFileSystem(project='ubc-mds-pool')


    for n, url in enumerate(complete_url.iloc[start_num:end_num, 0]):
        target_url = 'https://www.morphmarket.com' + url
        print(target_url)
        driver.get(target_url)
        time.sleep(4) # Loading page might take time
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        pic_container = soup.find_all('img', class_='thumbCarouselImage--RFBqw')
        pic_url = [img['srcset'].split(', ')[0].split(' ')[0] for img in pic_container if 'srcset' in img.attrs]
        if pic_url == []:
            pic_url = [soup.find_all('source')[0]['srcset'].split(' ')[0]]

        # Get traits
        traits = list(set([trait.get_text() for trait in soup.find_all('span', class_='labelBadge--oeqO_ dom-codom--KcmpO')]))

        # Get sex
        sex = 'unknown' # Default as unknown
        if soup.find_all('svg', class_='svg-inline--fa fa-mars sex male') != []:
            sex = 'male'
        if soup.find_all('svg', class_='svg-inline--fa fa-venus sex female') != []:
            sex = 'female'
        if soup.find_all('svg', class_='svg-inline--fa fa-venus-mars sex mixed') != []:
            sex = 'mixed'
        
        # Get price
        price = float(soup.find_all('h1', class_='salePrice--qNIIs')[0].get_text().replace("$", "").replace(",", ""))

        # Get birth
        try:
            birth_divs = [div for div in soup.find_all('div', class_='labelValueContainer--z1CP3')
                        if div.find('b', string='Birth:')]
            birth_dates = [div.find('span').text for div in birth_divs]
            birth = birth_dates[0]
        except:
            birth = ''

        # Download pictures
        for i, picture in enumerate(pic_url):
            pic_code = f'{n}-{i}'
            gcs_path = f'gs://sam_datastorage/scraper-morphmarket/data/img/{pic_code}.png'
            response = requests.get(picture)
            with fs.open(gcs_path, 'wb') as f:
                f.write(response.content)

            dict[pic_code] = [traits, sex, price, birth]
            print(f'{pic_code} completed')

        # Save result to csv every 5 pages
        if n % 5 == 0:
            labels_df = pd.DataFrame(dict, index=['traits', 'sex', 'price', 'birth']).T.reset_index()
            with fs.open('gs://sam_datastorage/scraper-morphmarket/data/labels.csv', 'w') as f:
                labels_df.to_csv(f)

    driver.quit()


In [13]:
result_dict = {}
scrape_pic(start_num=1, end_num=10)

https://www.morphmarket.com/us/c/reptiles/pythons/ball-pythons/1970200
0-0 completed
0-1 completed
https://www.morphmarket.com/us/c/reptiles/pythons/ball-pythons/933034
1-0 completed
1-1 completed
https://www.morphmarket.com/us/c/reptiles/pythons/ball-pythons/860291
2-0 completed
2-1 completed
2-2 completed
https://www.morphmarket.com/us/c/reptiles/pythons/ball-pythons/1669137
3-0 completed
https://www.morphmarket.com/us/c/reptiles/pythons/ball-pythons/1983770
4-0 completed
4-1 completed
4-2 completed
https://www.morphmarket.com/us/c/reptiles/pythons/ball-pythons/1833332
5-0 completed
https://www.morphmarket.com/us/c/reptiles/pythons/ball-pythons/1198102
6-0 completed
https://www.morphmarket.com/us/c/reptiles/pythons/ball-pythons/1983771
7-0 completed
7-1 completed
https://www.morphmarket.com/us/c/reptiles/pythons/ball-pythons/2074211
8-0 completed


In [31]:
dict = {'0-0': [['Pastel', 'Yellow Belly', 'Het Puzzle', '50% Het Hypo'], 'female', 'Self Produced', 650.0, '7th April 2023', 'https://www.morphmarket.com/us/c/reptiles/pythons/ball-pythons/1622444']}

In [40]:
labels_df = pd.DataFrame(dict, index=['genes', 'sex', 'origin', 'price', 'birth', 'url']).T
labels_df

,0-0
genes,"[Pastel, Yellow Belly, Het Puzzle, 50% Het Hypo]"
sex,female
origin,Self Produced
price,650.0
birth,7th April 2023
url,https://www.morphmarket.com/us/c/reptiles/pyth...
